In [169]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from scipy.sparse import coo_matrix, hstack
from sklearn.cluster import KMeans
import pickle
from collections import Counter
def savePickle(target, filename):
    with open(filename, "wb") as f:
        pickle.dump(target, f)
def loadPickle(filename):
    with open(filename, "rb") as f:
        return pickle.load(f)

 - 1_total_fee array([ 0.39827125,  0.39316978,  0.39571121])
 - 2_total_fee _array([ 0.34979964,  0.35166863,  0.35014922])
 - 3_total_fee array([ 0.34979964,  0.35166863,  0.35014922])
 - 4_total_fee  array([ 0.3525123 ,  0.35381964,  0.35444017])
 - 1234 array([ 0.56280605,  0.56499034,  0.56352438])
 c =  array([ 0.29369751,  0.29296579,  0.29410951])
 ---------------------------------------------------
 ####  total fee
  - 1234 0.5604
  - 1234 + 1/2 0.5630
  - 1234 +1/2 1/3 0.5648
  - 1234 + 1/2+1/3+2/4 0.5651
  - 1234 + 1/2 1/3 1/4 2/3 2/4 0.5662
  - 1234 + 1/2 1/3 1/4 2/3 2/4 1/234 0.5665
  - 1234 + 1/2 1/3 1/4 2/3 2/4 3/4 1/234 0.5674
  
#### traffic
  - fee + local 0.5994
  - fee + local + last 0.7095
  - fee + local _ last + month 0.7472
  - fee + local _ last + month +  last/month 0.7476
  - fee + local _ last + month +  last/month local/month 0.7493
  
#### call
  -  fee + traffic + local 0.7519
  -  fee + traffic + local + service1 0.7850
  -  fee + traffic + local + service1 + service2  0.8160
  -  fee + traffic + local + service1 + service2 + s2/local 0.8162
  
#### nature
  - fee + traffic + call + age  0.8184
  - fee + traffic + call + age + gender 0.8189
  
#### statistics
  - fee + traffic + call + age + onlinetime 0.0.8280
  - fee + traffic + call + age + onlinetime + service_type 0.8352
  - fee + traffic + call + age + onlinetime + service_type + is_mix_type 0.8362
  - ... +  many_over_bill 0.8443
  - ... +  contract_type 0.8496
  - ... + contract_time 0.8683
  - ... + is_promise_low_consume 0.8689
  - ... + pay_times 0.8690
  - ... + pay_num 0.8693
#### cross feature
  - ... + traffic/fee 0.8699
  - ... + traffic/call 0.8701
  - ... 8839， 0.8843

In [170]:
def square_f1_score(y_true, y_pred):
    y_pred = np.array(y_pred)
    y_true = np.array(y_true)
    samples = len(y_true)
    classes = int(len(y_pred)/samples)
    df = pd.DataFrame()
    for i in range(0, classes):
        df['c_%d'%i] =  y_pred[samples * i: samples*(i+1)]
    pred = np.argmax(np.array(df), axis = 1)
    score = np.square(f1_score(y_true, pred, average = 'macro'))
    return 'square_f1_score', score, True

In [264]:
enc = OneHotEncoder()
train = loadPickle("../data/train.pkl")
test = loadPickle("../data/test.pkl")
categoryes = sorted(train['current_service'].unique())
label_dict = {}
for i in range(len(categoryes)):
    label_dict[categoryes[i]] = i
    label_dict[i] = categoryes[i]

test['current_service'] = -1
train['current_service'] = np.array(train['current_service'].apply(lambda x:label_dict[x]))
train = train.append(test)
features = train[['1_total_fee', '2_total_fee', '3_total_fee', '4_total_fee']].copy()
features['1_2_total_fee_n'] = np.array(features['1_total_fee'])/np.array(features['2_total_fee'])
features['1_3_total_fee_n'] = np.array(features['1_total_fee'])/np.array(features['3_total_fee'])
features['2_4_total_fee_n'] = np.array(features['2_total_fee'])/np.array(features['4_total_fee'])
features['2_3_total_fee_n'] = np.array(features['2_total_fee'])/np.array(features['3_total_fee'])
features['3_4_total_fee_n'] = np.array(features['3_total_fee'])/np.array(features['4_total_fee'])
features['max_fee'] = np.max(np.array(features[['1_total_fee', '2_total_fee', '3_total_fee', '4_total_fee']]),axis = 1)
features['min_fee'] = np.min(np.array(features[['1_total_fee', '2_total_fee', '3_total_fee', '4_total_fee']]),axis = 1)
# traficc
features['local_trafffic_month'] = train['local_trafffic_month']
features['last_month_traffic'] = train['last_month_traffic']
features['month_traffic'] = train['month_traffic']
features['r_last_month_traffic'] = np.array(train['last_month_traffic'])/np.array(train['month_traffic'])
features['r_local_month_traffic'] = np.array(train['local_trafffic_month'])/np.array(train['month_traffic'])
features['non_local_trafffic_month'] = np.array(train['month_traffic']) - np.array(train['local_trafffic_month'])

#call
features['local_caller_time'] = train['local_caller_time']
features['service1_caller_time'] = train['service1_caller_time']
features['service2_caller_time'] = train['service2_caller_time']
features['s2_local_caller_time'] = np.array(train['service2_caller_time'])/np.array(train['local_caller_time'])
features['s1_s2_caller_time'] = np.array(train['service1_caller_time'])/np.array(train['service2_caller_time'])

#nature
features['age'] = train['age']
features['gender0'] = np.array(train['gender'] == 0).astype(int)
features['gender1'] = np.array(train['gender'] == 1).astype(int)
features['gender2'] = np.array(train['gender'] == 2).astype(int)

#other
features['online_time'] = train['online_time']
features['service_type'] = np.array(train['service_type'] == 4).astype(int)
features['service_type'] = np.array(train['service_type'] == 1).astype(int)
features['is_mix_service'] = train['is_mix_service']
features['many_over_bill'] = train['many_over_bill']
features['contract_type'] = np.array(train['contract_type'] == 0).astype(int)
features['contract_type'] = np.array(train['contract_type'] == 1).astype(int)
features['contract_type'] = np.array(train['contract_type'] == 2).astype(int)
features['contract_type'] = np.array(train['contract_type'] == 3).astype(int)
features['contract_type'] = np.array(train['contract_type'] == 6).astype(int)
features['contract_type'] = np.array(train['contract_type'] == 7).astype(int)
features['contract_type'] = np.array(train['contract_type'] == 8).astype(int)
features['contract_type'] = np.array(train['contract_type'] == 9).astype(int)
features['contract_type'] = np.array(train['contract_type'] == 8).astype(int)
features['contract_type'] = np.array(train['contract_type'] == 12).astype(int)
for t in sorted(train['contract_time'].unique()):
    features['contract_time_%d'%t] = np.array(train['contract_time'] == t).astype(int)
features['is_promise_low_consume'] = train['is_promise_low_consume']
# for t in sorted(train['net_service'].unique()):
#     features['net_service_%d'%t] = np.array(train['net_service'] == t).astype(int) not good
features['pay_times'] = train['pay_times']
features['pay_num'] = train['pay_num']

# cross
features['traffic_fee'] = np.array(train['month_traffic'])/np.array(train['1_total_fee'])

# userid 
features['user_id'] = train['user_id']
features['current_service'] = train['current_service']
features = features.fillna(-1)
train = features[features.current_service >= 0].copy()
test = features[features.current_service < 0].drop(['current_service'], axis = 1).copy()

/usr/local/lib/python3.6/site-packages/pandas/core/frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in true_divide
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in true_divide
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divid

In [314]:
du = train[train.duplicated(keep = False)]
# du = du[du['current_service'] != 3]
# du = du[du['current_service'] != 4]
train_x = train.drop_duplicates(keep = False).drop(['current_service', 'user_id'], axis = 1)
train_y = np.array(train.drop_duplicates(keep = False)['current_service'])
du_x = du.drop(['current_service', 'user_id'], axis = 1)
du_y = np.array(du['current_service'])
train_x = np.vstack([train_x, du_x])
train_y= np.hstack([train_y, du_y])

In [315]:
train_x, eval_x, train_y, eval_y = train_test_split(train_x, train_y, test_size = 0.1, random_state = 50)

In [316]:
# train_x = np.array(train.drop(['current_service', 'user_id'], axis = 1))
# train_y = np.array(train['current_service'])

In [317]:
#early_stopping_rounds = 50,

In [ ]:
weight_dict = {0:1, 1:1, 2:1, 3:0.7, 4:0.9, 5:1, 6:1, 7:2, 8:2, 9:2, 10:2}
lgbm = lgb.LGBMClassifier(objective = "multiclass", boosting_type =  'gbdt', class_weight = None,
                          colsample_bytree = 0.625, importance_type = 'split', learning_rate = 0.2, max_depth = 9,
                         min_child_samples = 133, min_child_weight = 2, min_split_gain = 0.1, n_estimators = 1650, n_jobs = -1,
                         num_leaves = 58, random_state = 10, reg_alpha = 0.0005, reg_lambda = 0.0471,  subsample = 0.9375,
                        subsample_for_bin = 1650, subsample_freq = 1,verbosity = 0, verbose_eval = 100)
lgbm.fit(train_x, train_y, eval_set = (eval_x, eval_y),  eval_metric = square_f1_score, early_stopping_rounds = None)

[1]	valid_0's multi_logloss: 1.62086	valid_0's square_f1_score: 0.60959
[2]	valid_0's multi_logloss: 1.30797	valid_0's square_f1_score: 0.640663
[3]	valid_0's multi_logloss: 1.09042	valid_0's square_f1_score: 0.667864
[4]	valid_0's multi_logloss: 0.930148	valid_0's square_f1_score: 0.685719
[5]	valid_0's multi_logloss: 0.809785	valid_0's square_f1_score: 0.695808
[6]	valid_0's multi_logloss: 0.714352	valid_0's square_f1_score: 0.703155
[7]	valid_0's multi_logloss: 0.637217	valid_0's square_f1_score: 0.708748
[8]	valid_0's multi_logloss: 0.574234	valid_0's square_f1_score: 0.713351
[9]	valid_0's multi_logloss: 0.523331	valid_0's square_f1_score: 0.71724
[10]	valid_0's multi_logloss: 0.479843	valid_0's square_f1_score: 0.721781
[11]	valid_0's multi_logloss: 0.443667	valid_0's square_f1_score: 0.725344
[12]	valid_0's multi_logloss: 0.413534	valid_0's square_f1_score: 0.728562
[13]	valid_0's multi_logloss: 0.388861	valid_0's square_f1_score: 0.730655
[14]	valid_0's multi_logloss: 0.368049	

[111]	valid_0's multi_logloss: 0.196315	valid_0's square_f1_score: 0.791787
[112]	valid_0's multi_logloss: 0.196091	valid_0's square_f1_score: 0.791321
[113]	valid_0's multi_logloss: 0.195928	valid_0's square_f1_score: 0.791738
[114]	valid_0's multi_logloss: 0.195703	valid_0's square_f1_score: 0.791834
[115]	valid_0's multi_logloss: 0.195519	valid_0's square_f1_score: 0.792554
[116]	valid_0's multi_logloss: 0.195336	valid_0's square_f1_score: 0.792612
[117]	valid_0's multi_logloss: 0.195169	valid_0's square_f1_score: 0.792412
[118]	valid_0's multi_logloss: 0.194966	valid_0's square_f1_score: 0.792925
[119]	valid_0's multi_logloss: 0.194753	valid_0's square_f1_score: 0.793276
[120]	valid_0's multi_logloss: 0.19453	valid_0's square_f1_score: 0.793828
[121]	valid_0's multi_logloss: 0.194341	valid_0's square_f1_score: 0.793586
[122]	valid_0's multi_logloss: 0.194086	valid_0's square_f1_score: 0.7944
[123]	valid_0's multi_logloss: 0.193918	valid_0's square_f1_score: 0.794771
[124]	valid_0's

[220]	valid_0's multi_logloss: 0.182619	valid_0's square_f1_score: 0.806338
[221]	valid_0's multi_logloss: 0.182537	valid_0's square_f1_score: 0.806467
[222]	valid_0's multi_logloss: 0.182493	valid_0's square_f1_score: 0.806279
[223]	valid_0's multi_logloss: 0.182442	valid_0's square_f1_score: 0.806478
[224]	valid_0's multi_logloss: 0.182352	valid_0's square_f1_score: 0.806973
[225]	valid_0's multi_logloss: 0.182289	valid_0's square_f1_score: 0.807634
[226]	valid_0's multi_logloss: 0.182183	valid_0's square_f1_score: 0.807497
[227]	valid_0's multi_logloss: 0.182107	valid_0's square_f1_score: 0.807954
[228]	valid_0's multi_logloss: 0.182035	valid_0's square_f1_score: 0.807681
[229]	valid_0's multi_logloss: 0.181963	valid_0's square_f1_score: 0.808195
[230]	valid_0's multi_logloss: 0.181904	valid_0's square_f1_score: 0.808022
[231]	valid_0's multi_logloss: 0.181843	valid_0's square_f1_score: 0.80824
[232]	valid_0's multi_logloss: 0.181744	valid_0's square_f1_score: 0.808258
[233]	valid_0

[329]	valid_0's multi_logloss: 0.175306	valid_0's square_f1_score: 0.817984
[330]	valid_0's multi_logloss: 0.17527	valid_0's square_f1_score: 0.81824
[331]	valid_0's multi_logloss: 0.175212	valid_0's square_f1_score: 0.818123
[332]	valid_0's multi_logloss: 0.175153	valid_0's square_f1_score: 0.818261
[333]	valid_0's multi_logloss: 0.175107	valid_0's square_f1_score: 0.818233
[334]	valid_0's multi_logloss: 0.175071	valid_0's square_f1_score: 0.818395
[335]	valid_0's multi_logloss: 0.175018	valid_0's square_f1_score: 0.81843
[336]	valid_0's multi_logloss: 0.174986	valid_0's square_f1_score: 0.818231
[337]	valid_0's multi_logloss: 0.174917	valid_0's square_f1_score: 0.818337
[338]	valid_0's multi_logloss: 0.174864	valid_0's square_f1_score: 0.818449
[339]	valid_0's multi_logloss: 0.174823	valid_0's square_f1_score: 0.818309
[340]	valid_0's multi_logloss: 0.174745	valid_0's square_f1_score: 0.818367
[341]	valid_0's multi_logloss: 0.174668	valid_0's square_f1_score: 0.818291
[342]	valid_0's

[438]	valid_0's multi_logloss: 0.170318	valid_0's square_f1_score: 0.824342
[439]	valid_0's multi_logloss: 0.170274	valid_0's square_f1_score: 0.824334
[440]	valid_0's multi_logloss: 0.170253	valid_0's square_f1_score: 0.824449
[441]	valid_0's multi_logloss: 0.1702	valid_0's square_f1_score: 0.824313
[442]	valid_0's multi_logloss: 0.17017	valid_0's square_f1_score: 0.824238
[443]	valid_0's multi_logloss: 0.170137	valid_0's square_f1_score: 0.824576
[444]	valid_0's multi_logloss: 0.170091	valid_0's square_f1_score: 0.824591
[445]	valid_0's multi_logloss: 0.170059	valid_0's square_f1_score: 0.824599
[446]	valid_0's multi_logloss: 0.170022	valid_0's square_f1_score: 0.824843
[447]	valid_0's multi_logloss: 0.170005	valid_0's square_f1_score: 0.824601
[448]	valid_0's multi_logloss: 0.169956	valid_0's square_f1_score: 0.824739
[449]	valid_0's multi_logloss: 0.169903	valid_0's square_f1_score: 0.824569
[450]	valid_0's multi_logloss: 0.16986	valid_0's square_f1_score: 0.824576
[451]	valid_0's 

[546]	valid_0's multi_logloss: 0.166874	valid_0's square_f1_score: 0.828008
[547]	valid_0's multi_logloss: 0.166834	valid_0's square_f1_score: 0.82824
[548]	valid_0's multi_logloss: 0.16681	valid_0's square_f1_score: 0.828623
[549]	valid_0's multi_logloss: 0.166802	valid_0's square_f1_score: 0.828338
[550]	valid_0's multi_logloss: 0.166749	valid_0's square_f1_score: 0.828495
[551]	valid_0's multi_logloss: 0.1667	valid_0's square_f1_score: 0.828633
[552]	valid_0's multi_logloss: 0.166679	valid_0's square_f1_score: 0.828478
[553]	valid_0's multi_logloss: 0.166664	valid_0's square_f1_score: 0.82829
[554]	valid_0's multi_logloss: 0.166637	valid_0's square_f1_score: 0.828486
[555]	valid_0's multi_logloss: 0.166599	valid_0's square_f1_score: 0.828542
[556]	valid_0's multi_logloss: 0.166591	valid_0's square_f1_score: 0.828491
[557]	valid_0's multi_logloss: 0.166558	valid_0's square_f1_score: 0.828697
[558]	valid_0's multi_logloss: 0.166542	valid_0's square_f1_score: 0.82886
[559]	valid_0's mu

[655]	valid_0's multi_logloss: 0.164624	valid_0's square_f1_score: 0.832124
[656]	valid_0's multi_logloss: 0.16462	valid_0's square_f1_score: 0.832047
[657]	valid_0's multi_logloss: 0.164616	valid_0's square_f1_score: 0.832237
[658]	valid_0's multi_logloss: 0.164585	valid_0's square_f1_score: 0.832221
[659]	valid_0's multi_logloss: 0.164572	valid_0's square_f1_score: 0.832505
[660]	valid_0's multi_logloss: 0.164545	valid_0's square_f1_score: 0.832609
[661]	valid_0's multi_logloss: 0.164545	valid_0's square_f1_score: 0.832461
[662]	valid_0's multi_logloss: 0.164519	valid_0's square_f1_score: 0.832323
[663]	valid_0's multi_logloss: 0.164483	valid_0's square_f1_score: 0.832165
[664]	valid_0's multi_logloss: 0.164472	valid_0's square_f1_score: 0.832394
[665]	valid_0's multi_logloss: 0.164455	valid_0's square_f1_score: 0.83242
[666]	valid_0's multi_logloss: 0.164436	valid_0's square_f1_score: 0.832552
[667]	valid_0's multi_logloss: 0.164412	valid_0's square_f1_score: 0.832463
[668]	valid_0'

[764]	valid_0's multi_logloss: 0.162999	valid_0's square_f1_score: 0.83351
[765]	valid_0's multi_logloss: 0.162993	valid_0's square_f1_score: 0.833641
[766]	valid_0's multi_logloss: 0.162983	valid_0's square_f1_score: 0.833574
[767]	valid_0's multi_logloss: 0.16297	valid_0's square_f1_score: 0.833712
[768]	valid_0's multi_logloss: 0.162946	valid_0's square_f1_score: 0.833616
[769]	valid_0's multi_logloss: 0.162938	valid_0's square_f1_score: 0.833485
[770]	valid_0's multi_logloss: 0.162918	valid_0's square_f1_score: 0.833511
[771]	valid_0's multi_logloss: 0.162869	valid_0's square_f1_score: 0.833452
[772]	valid_0's multi_logloss: 0.16286	valid_0's square_f1_score: 0.833385
[773]	valid_0's multi_logloss: 0.162857	valid_0's square_f1_score: 0.833565
[774]	valid_0's multi_logloss: 0.162858	valid_0's square_f1_score: 0.833521
[775]	valid_0's multi_logloss: 0.162846	valid_0's square_f1_score: 0.833492
[776]	valid_0's multi_logloss: 0.162834	valid_0's square_f1_score: 0.833494
[777]	valid_0's

In [248]:
# [336]	valid_0's multi_logloss: 0.186802	valid_0's square_f1_score: 0.816288
# [337]	valid_0's multi_logloss: 0.186818	valid_0's square_f1_score: 0.816364
# [338]	valid_0's multi_logloss: 0.186843	valid_0's square_f1_score: 0.816314
# Early stopping, best iteration is:
# [288]	valid_0's multi_logloss: 0.18625	valid_0's square_f1_score: 0.814733

In [296]:
pred = lgbm.predict(eval_x)
score = f1_score(eval_y, pred, average = 'macro')
print(score, np.square(score))

0.9158720353299828 0.8388215850994853


/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [297]:
from sklearn.metrics import classification_report

In [306]:
test_feature = test.drop(['user_id'], axis = 1)
TestResult = test[['user_id']].copy()
TestResult['predict']  = lgbm.predict(test_feature)
TestResult['predict'] = TestResult['predict'].apply(lambda x: label_dict[x])

/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [307]:
TestResult.to_csv("../lgb8404.csv", index = None)

In [308]:
complex_pred = pd.read_csv("../lgb_xgb_complex_co1008.csv")
xgb_pred = pd.read_csv("../xgb_ox_2799743984766_0.8280.csv")
print(np.sum(TestResult.predict == complex_pred.predict))
print(np.sum(TestResult.predict == xgb_pred.predict))
print(np.sum(complex_pred.predict == xgb_pred.predict))

194973
194730
198897


In [ ]:
195089
194867
198897

In [302]:
print(classification_report(eval_y, pred, digits = 3))
print(label_dict, end = "\n\n")

             precision    recall  f1-score   support

          0      0.933     0.943     0.938     13332
          1      0.921     0.885     0.902      7472
          2      0.980     0.940     0.959      3312
          3      0.998     0.998     0.998     28559
          4      0.995     0.999     0.997      3775
          5      0.972     0.975     0.974      2188
          6      0.953     0.960     0.957      2021
          7      0.829     0.883     0.855      2941
          8      0.805     0.821     0.813      3257
          9      0.854     0.871     0.862      5404
         10      0.839     0.799     0.819      2138

avg / total      0.946     0.945     0.945     74399

{89950166: 0, 0: 89950166, 89950167: 1, 1: 89950167, 89950168: 2, 2: 89950168, 90063345: 3, 3: 90063345, 90109916: 4, 4: 90109916, 90155946: 5, 5: 90155946, 99999825: 6, 6: 99999825, 99999826: 7, 7: 99999826, 99999827: 8, 8: 99999827, 99999828: 9, 9: 99999828, 99999830: 10, 10: 99999830}



In [206]:
precision    recall  f1-score   support

          0       0.93      0.93      0.93     40270
          1       0.91      0.86      0.89     22125
          2       0.97      0.93      0.95     10030
          3       1.00      1.00      1.00     85873
          4       1.00      1.00      1.00     11329
          5       0.96      0.97      0.97      6665
          6       0.95      0.95      0.95      6114
          7       0.81      0.86      0.83      8738
          8       0.78      0.80      0.79      9679
          9       0.82      0.85      0.84     15940
         10       0.81      0.78      0.79      6433

avg / total       0.94      0.94      0.94    223196

precision    recall  f1-score   support

          0       0.92      0.94      0.93     40270
          1       0.89      0.88      0.89     22125
          2       0.97      0.93      0.95     10030
          3       1.00      1.00      1.00     85873
          4       1.00      1.00      1.00     11329
          5       0.97      0.97      0.97      6665
          6       0.94      0.96      0.95      6114
          7       0.81      0.85      0.83      8738
          8       0.78      0.78      0.78      9679
          9       0.84      0.83      0.83     15940
         10       0.83      0.74      0.78      6433

avg / total       0.94      0.94      0.94    223196



SyntaxError: invalid syntax (<ipython-input-206-d916815212cd>, line 1)

In [150]:
#sorted(dict(zip(du_x.columns, lgbm.feature_importances_)).items(), key = lambda x:x[1])

In [235]:

def print_stats(tdf):
    dct = Counter(tdf)
    cn = len(tdf)
    for key in sorted(dct.keys()):
        print(key, round(dct[key]/cn, 3))
    print(cn, end = "\n\n")
print_stats(train_y)
print_stats(TestResult.predict)
print_stats(xgb_pred.predict)
print_stats(complex_pred.predict)

0 0.178
1 0.099
2 0.045
3 0.387
4 0.051
5 0.03
6 0.027
7 0.039
8 0.044
9 0.071
10 0.028
520790

89950166 0.084
89950167 0.101
89950168 0.019
90063345 0.58
90109916 0.05
90155946 0.035
99999825 0.015
99999826 0.026
99999827 0.022
99999828 0.045
99999830 0.023
200000

89950166 0.085
89950167 0.102
89950168 0.019
90063345 0.58
90109916 0.05
90155946 0.035
99999825 0.016
99999826 0.028
99999827 0.021
99999828 0.044
99999830 0.021
200000

89950166 0.086
89950167 0.102
89950168 0.019
90063345 0.58
90109916 0.05
90155946 0.035
99999825 0.016
99999826 0.028
99999827 0.021
99999828 0.044
99999830 0.02
200000



In [17]:
# kmeans1 = KMeans(n_clusters=10,  n_jobs = -1, random_state = 2018).fit(train_x)
# train_c = kmeans1.predict(train_x)
# train_c = train_c.reshape(-1,1)
# train_c = enc.fit_transform(train_c)
# train_co = hstack((train_x, train_c))

In [19]:
print(np.mean(scores))
print(np.square(np.mean(scores)))

0.883655080019
0.780846300444


In [ ]:
#lgbm.fit(train_x,train_y)

In [697]:
sorted(dict(zip(features.columns, lgbm.feature_importances_)).items(),key=lambda x:x[1])

NotFittedError: No feature_importances found. Need to call fit beforehand.

In [ ]:
plt.plot(sorted(train[train['current_service'] == 99999825]['contract_time']))

In [ ]:
plt.plot(sorted(train[train['current_service'] == 99999826]['online_time']))

In [ ]:
sorted(train['online_time'].unique())

In [ ]:
features.columns

In [ ]:
train.user_id

In [437]:
lgbm.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': 'multiclass',
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': True,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 1}